In [49]:
import os
import re
import random
import json
from tqdm import tqdm
import pickle
import pandas as pd
import numpy as np
from dotenv import load_dotenv

from abc import ABC, abstractmethod
from typing import List, Optional, Tuple, Dict, Union

import transformers
import torch
import torch.nn.functional as F

import openai
import datasets
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset, load_dataset

import plotly.graph_objects as go
import plotly.express as px

from utils import untuple, eval_completions
from scripts.get_activations import gen_pile_data, compare_token_lists, slice_acts

from act_add.model_wrapper import ModelWrapper
from act_add.rep_reader import RepReader, CAARepReader, PCARepReader
from act_add.contrast_dataset import ContrastDataset
from act_add.steering_pipeline import SteeringPipeline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
from datasets import load_from_disk

model_name_or_path = "meta-llama/Llama-2-7b-hf"

model = AutoModelForCausalLM.from_pretrained(model_name_or_path, torch_dtype=torch.float16, device_map="auto").eval()
use_fast_tokenizer = "LlamaForCausalLM" not in model.config.architectures
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast = use_fast_tokenizer, padding_side="left")
tokenizer.pad_token_id = 0 if tokenizer.pad_token_id is None else tokenizer.pad_token_id
tokenizer.bos_token_id = 1

llama_mem_data = pd.read_csv("data/llama-2-7b/llama_ground_data.csv")
llama_states = torch.load("data/llama-2-7b/all_hidden_states.pt")

path = 'data/llama-2-7b'
dataset = load_from_disk(os.path.join(path, 'hf_dataset_split'))

mw = ModelWrapper(model = model, tokenizer = tokenizer)

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.68s/it]


# mem -> unmem

In [86]:
def is_label_one(example):
    return example['label'] == 1

memmed_test_samples = dataset['test'].filter(is_label_one)

def gen_eval_data(ground, tokenizer, input_length = 32, max_length = 64):
    if isinstance(ground[0], str):
        tokens = tokenizer(ground, padding = True, truncation = True, max_length = max_length, return_tensors = 'pt')
        inputs = tokenizer.batch_decode(tokens['input_ids'][:, :input_length])
        targets = tokenizer.batch_decode(tokens['input_ids'][:, input_length:])
    else:
        inputs = tokenizer.batch_decode(ground[:, :input_length], skip_special_tokens = True)
        targets = tokenizer.batch_decode(ground[:, input_length:])
    return inputs, targets

ground_toks = [eval(toks) for toks in memmed_test_samples['ground_llama_toks']]
inputs, targets = gen_eval_data(torch.tensor(ground_toks), tokenizer)

In [87]:
def get_token_states(states, dataset, tok_idxs, layer):
    assert len(dataset) == states.shape[0], "dataset and states must have the same number of rows"
    
    y = torch.from_numpy(np.array([[label] * len(tok_idxs) for label in dataset['label']]).flatten())

    new_X = []
    for x in states:
        new_X.append(torch.stack([x[layer, tok_idx] for tok_idx in tok_idxs]))
    return torch.cat(new_X, dim = 0).float(), y.float()

## CAA

In [43]:
mem_rep_reader = CAARepReader()
mw = ModelWrapper(model = model, tokenizer = tokenizer)

mem_steering_pipeline = SteeringPipeline(mw, None, mem_rep_reader)

In [44]:
N = 2995
TOKEN_IDXS = [5, 6, 7, 8, 9]
N_LAYERS = model.config.num_hidden_layers
D_M = model.config.hidden_size
rep_token_idx = -1
hidden_layers = list(range(N_LAYERS))
n_difference = None
train_labels = None

train_states = llama_states[dataset['train']['index_in_states']]
X_train, y_train = get_token_states(train_states, dataset['train'], TOKEN_IDXS, hidden_layers)

mem_X = X_train[torch.where(y_train == 1)].reshape(N_LAYERS, -1, D_M)[:, :N]
unmem_X = X_train[torch.where(y_train == 0)].reshape(N_LAYERS, -1, D_M)[:, :N]

In [45]:
dirs = mem_steering_pipeline.gen_dir_from_states(mem_X, unmem_X, hidden_layers, n_difference, train_labels)

In [46]:
norms = [np.linalg.norm(dirs[key]) for key in dirs]

fig = go.Figure(data=go.Scatter(x=list(range(len(norms))), y=norms, mode='lines'))
fig.update_layout(title='Norms of Dirs', xaxis_title='Index', yaxis_title='Norm')
fig.show()


In [48]:
# layer_id = list(range(15,25))
layer_id = [0,1,3,4,5]
coeff=.05

# layer_id = [0,1,3,4,5,6,23,24,25]
# coeff = 0.07

batch_size=50

max_new_tokens=32

print(f"Coeff: {coeff}")
print(f"LAYERS: {layer_id}")
print("RepReader:")
print("No Control")
baseline_outputs = mem_steering_pipeline.batch_steering_generate(inputs, 
                                                                layer_id, 
                                                                coeff = 0 * coeff, 
                                                                batch_size = batch_size, 
                                                                use_tqdm=True, 
                                                                max_new_tokens=max_new_tokens)

print(eval_completions(baseline_outputs, targets))

print("+ Memorization")
pos_outputs = mem_steering_pipeline.batch_steering_generate(inputs, 
                                                            layer_id, 
                                                            coeff = coeff, 
                                                            batch_size = batch_size, 
                                                            use_tqdm=True, 
                                                            max_new_tokens=max_new_tokens)
print(eval_completions(pos_outputs, targets))

print("- Memorization")
neg_outputs = mem_steering_pipeline.batch_steering_generate(inputs, 
                                                            layer_id, 
                                                            coeff = -coeff, 
                                                            batch_size = batch_size, 
                                                            use_tqdm=True, 
                                                            max_new_tokens=max_new_tokens)
print(eval_completions(neg_outputs, targets))

Coeff: 0.05
LAYERS: [0, 1, 3, 4, 5]
RepReader:
No Control


  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:26<00:00,  4.49s/it]


{'char_by_char_similarity': 0.8645670658441992, 'sem_similarity': 0.942746433093893, 'lev_distance': 0.9110144725331148}
+ Memorization


100%|██████████| 6/6 [00:26<00:00,  4.44s/it]


{'char_by_char_similarity': 0.8217848330530468, 'sem_similarity': 0.9380261842098702, 'lev_distance': 0.8881985010836166}
- Memorization


100%|██████████| 6/6 [00:26<00:00,  4.44s/it]


{'char_by_char_similarity': 0.7535083938712123, 'sem_similarity': 0.9082607215000724, 'lev_distance': 0.8381929592368699}


In [34]:
pos_outputs[:10]

['ATMSGMSG\nMSGMSGMSGMSGMSGMSGMSGMSGMSGMSGMSGMSGMSG',
 '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n',
 '\n\n\t\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n',
 '\r\nTIT\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n',
 'A;\n\n\n                )\n\n\n *\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n',
 '\n            if (C)\n            ;\n            NOTICEGREA\n            ==CMSGSA\n           \n           \n            INSTG',
 '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n',
 '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n',
 'GMSGAA\nMSG\n\n\n\n\n\n\n\n\n\nI here\n\n\n\n\n\n\n\n\n\n\n\n\n',
 'VIDC",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n']

In [120]:
neg_outputs[:10]

['ARTicular Usage, Or The Occupation Of A Specific)\nWas also Included\n)\n[ ]\nThe "The Wrong)',
 '\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t',
 '33</string>\n\t\t<string>m0434</cym 434>\n\t\t<string>64',
 '-1898-1999, 1999+" InMinimum Reyes">>"-" 199',
 'ied by 10);\n                case Deccasapr: return this.plusYears(10);\n                case: return this.',
 ' true;\n            ConfigurePasswordGenerator;\n            // For the first time, pass the creator of the\n            // passwords to the call.\n',
 ' You may obtain a copy of this file by the following methods:\n#  (1) The Internet and writing "nh 11111',
 '\n//    1. The Software and any copies thereof may be used only on\n//       devices, and by persons, who are registered to use them',
 'PECIAL, EXCEPTION *\n * OR EXCUSsior FOR a thcir FOR a thru Excruse 1',
 'CURE, EROth, ERerver, EOen not en, EOen not en, EOen not en, EO']

In [26]:
inputs = inputs[:100]
targets = targets[:100]

In [27]:
pos_levs = []
neg_levs = []
coeff = 0.1
max_new_tokens = 32
batch_size = 50
for layer in tqdm(range(mw.model.config.num_hidden_layers)):
    layer_id = [layer]
    # print("+ Memorization")
    pos_outputs = mem_steering_pipeline.batch_steering_generate(inputs, 
                                                                layer_id, 
                                                                coeff = coeff, 
                                                                batch_size = batch_size, 
                                                                use_tqdm=False, 
                                                                max_new_tokens=max_new_tokens)
    pos_res = eval_completions(pos_outputs, targets)

    # print("- Memorization")
    neg_outputs = mem_steering_pipeline.batch_steering_generate(inputs, 
                                                                layer_id, 
                                                                coeff = -coeff, 
                                                                batch_size = batch_size, 
                                                                use_tqdm=False, 
                                                                max_new_tokens=max_new_tokens)
    neg_res = eval_completions(neg_outputs, targets)
    
    pos_levs.append(pos_res['lev_distance'])
    neg_levs.append(neg_res['lev_distance'])

100%|██████████| 32/32 [11:41<00:00, 21.92s/it]


In [28]:
import plotly.express as px

fig = px.line(y=pos_levs, title="Positive and Negative Memorization")
fig.add_scatter(y=neg_levs, name="Negative Memorization")
fig.show()


## probe intervention

In [54]:
#* load data
from probes import LRProbe
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from sklearn.linear_model import LogisticRegression
from datasets import load_from_disk, DatasetDict

LAYER = 10
TOK_IDXS = [5, 6, 7, 8, 9]

train_states = llama_states[dataset['train']['index_in_states']]
val_states = llama_states[dataset['val']['index_in_states']]
test_states = llama_states[dataset['test']['index_in_states']]

def get_token_states(states, dataset, tok_idxs, layer):
    assert len(dataset) == states.shape[0], "dataset and states must have the same number of rows"
    
    y = torch.from_numpy(np.array([[label] * len(tok_idxs) for label in dataset['label']]).flatten())

    new_X = []
    for x in states:
        new_X.append(torch.stack([x[layer, tok_idx] for tok_idx in tok_idxs]))
    return torch.cat(new_X, dim = 0).float(), y.float()

X_train, y_train = get_token_states(train_states, dataset['train'], TOK_IDXS, LAYER)
X_val, y_val = get_token_states(val_states, dataset['val'], TOK_IDXS, LAYER)
X_test, y_test = get_token_states(test_states, dataset['test'], TOK_IDXS, LAYER)

In [6]:
lr = 0.01
weight_decay = 10
epochs = 500
use_bias = True
normalize = False

probe = LRProbe.from_data(X_train, y_train, 
                          lr = lr, 
                          weight_decay = weight_decay, 
                          epochs = epochs, 
                          use_bias = use_bias,
                          device = "cuda", )

acc = probe.get_probe_accuracy(X_val, y_val, device = "cuda")
auc = probe.get_probe_auc(X_val, y_val, device = "cuda")

print(f"PROBE LAYER {LAYER} TOKEN {TOK_IDXS}")
print(f"Accuracy {acc}")
print(f"AUC {auc}")
print()

PROBE LAYER 10 TOKEN [5, 6, 7, 8, 9]
Accuracy 0.8730000257492065
AUC 0.938079231692677



In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# Create an instance of Logistic Regression
probe_lr = LogisticRegression(max_iter = 5000, C = 1e-1)

# Train the probe using the training data
probe_lr.fit(X_train.numpy(), y_train.numpy())

# Predict the labels for X_val
y_pred = probe_lr.predict(X_val.numpy())

# Calculate the accuracy
accuracy = accuracy_score(y_val.numpy(), y_pred)

# Calculate the AUC
auc = roc_auc_score(y_val.numpy(), probe_lr.predict_proba(X_val.numpy())[:, 1])

accuracy, auc

(0.88, 0.945514205682273)

In [59]:
from probes import LRProbe
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

probes = []
accs = []
aucs = []
TOK_IDXS = [5, 6, 7, 8, 9]

for layer in tqdm(range(mw.model.config.num_hidden_layers)):
    X_train, y_train = get_token_states(train_states, dataset['train'], TOK_IDXS, layer)
    X_val, y_val = get_token_states(val_states, dataset['val'], TOK_IDXS, layer)
    X_test, y_test = get_token_states(test_states, dataset['test'], TOK_IDXS, layer)

    probe_lr = LogisticRegression(max_iter = 5000, C = 1e-3, fit_intercept = False)

    # Train the probe using the training data
    probe_lr.fit(X_train.numpy(), y_train.numpy())

    # Predict the labels for X_val
    y_pred = probe_lr.predict(X_val.numpy())

    # Calculate the accuracy
    accuracy = accuracy_score(y_val.numpy(), y_pred)

    # Calculate the AUC
    auc = roc_auc_score(y_val.numpy(), probe_lr.predict_proba(X_val.numpy())[:, 1])

    probes.append(probe_lr)
    accs.append(accuracy)
    aucs.append(auc)

  0%|          | 0/32 [00:00<?, ?it/s]

100%|██████████| 32/32 [00:15<00:00,  2.11it/s]


In [60]:
fig = px.line(y=accs, title="Accuracy of LR Probes")
fig.add_scatter(y=aucs, name="AUC of LR Probes")
fig.show()

In [71]:
import numpy as np
import plotly.graph_objects as go

norms = [np.linalg.norm(probe.coef_) for probe in probes]

fig = go.Figure(data=go.Scatter(x=list(range(len(norms))), y=norms, mode='lines'))
fig.update_layout(title='Norms of Probe Coefficients', xaxis_title='Index', yaxis_title='Norm')
fig.show()


In [64]:
from act_add.rep_reader import ProbeRepReader

probe_weight_dict = {}
for layer in range(mw.model.config.num_hidden_layers):
    probe_lr = probes[layer]
    probe_weight_dict[layer] = torch.from_numpy(probe_lr.coef_[0]) / torch.norm(torch.from_numpy(probe_lr.coef_[0]), p = 2)

probe_rep_reader = ProbeRepReader(probe_weight_dict)

In [65]:
mw = ModelWrapper(model = model, tokenizer = tokenizer)

mem_steering_pipeline = SteeringPipeline(mw, None, probe_rep_reader)

In [88]:
# layer_id = list(range(15,25))
from utils import eval_completions
layer_id = list(range(11, 21))
coeff= 1.75 # tune this parameter

# layer_id = list(range(1, 4)) + [7, 9] + list(range(11,21)) 
# coeff = 1

batch_size=50
max_new_tokens=32

print(f"Coeff: {coeff}")
print(f"LAYERS: {layer_id}")
print("RepReader:")
print("No Control")
baseline_outputs = mem_steering_pipeline.batch_steering_generate(inputs, 
                                                                layer_id, 
                                                                coeff = 0 * coeff, 
                                                                batch_size = batch_size, 
                                                                use_tqdm=True, 
                                                                operator = "linear_comb",
                                                                max_new_tokens=max_new_tokens,
                                                                top_p = 1.0,)

print(eval_completions(baseline_outputs, targets))

print("+ Memorization")
pos_outputs = mem_steering_pipeline.batch_steering_generate(inputs, 
                                                            layer_id, 
                                                            coeff = coeff, 
                                                            batch_size = batch_size, 
                                                            use_tqdm=True, 
                                                            operator = "linear_comb",                                    
                                                            max_new_tokens=max_new_tokens,
                                                            top_p = 1.0,)
print(eval_completions(pos_outputs, targets))

print("- Memorization")
neg_outputs = mem_steering_pipeline.batch_steering_generate(inputs, 
                                                            layer_id, 
                                                            coeff = -coeff, 
                                                            batch_size = batch_size, 
                                                            use_tqdm=True, 
                                                            operator = "linear_comb",
                                                            max_new_tokens=max_new_tokens,
                                                            top_p = 1.0,)
print(eval_completions(neg_outputs, targets))

Coeff: 1.75
LAYERS: [11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
RepReader:
No Control


  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:26<00:00,  4.44s/it]


{'char_by_char_similarity': 0.8231955711180643, 'sem_similarity': 0.9266210285576706, 'lev_distance': 0.8878613419393594}
+ Memorization


100%|██████████| 6/6 [00:27<00:00,  4.59s/it]


{'char_by_char_similarity': 0.5438279212353516, 'sem_similarity': 0.8173184714433721, 'lev_distance': 0.6909637039956827}
- Memorization


100%|██████████| 6/6 [00:26<00:00,  4.41s/it]


{'char_by_char_similarity': 0.3756710326817498, 'sem_similarity': 0.7178534652147229, 'lev_distance': 0.5219709667751323}


In [74]:
pos_levs = []
neg_levs = []
coeff = 2
max_new_tokens = 32
batch_size = 50
for layer in tqdm(range(mw.model.config.num_hidden_layers)):
    layer_id = [layer]
    # print("+ Memorization")
    pos_outputs = mem_steering_pipeline.batch_steering_generate(inputs, 
                                                                layer_id, 
                                                                coeff = coeff, 
                                                                batch_size = batch_size, 
                                                                use_tqdm=False, 
                                                                max_new_tokens=max_new_tokens)
    pos_res = eval_completions(pos_outputs, targets)

    # print("- Memorization")
    neg_outputs = mem_steering_pipeline.batch_steering_generate(inputs, 
                                                                layer_id, 
                                                                coeff = -coeff, 
                                                                batch_size = batch_size, 
                                                                use_tqdm=False, 
                                                                max_new_tokens=max_new_tokens)
    neg_res = eval_completions(neg_outputs, targets)
    
    pos_levs.append(pos_res['lev_distance'])
    neg_levs.append(neg_res['lev_distance'])

100%|██████████| 32/32 [11:44<00:00, 22.01s/it]


In [75]:
fig = px.line(y=pos_levs, title="Positive and Negative Memorization")
fig.add_scatter(y=neg_levs, name="Negative Memorization")
fig.show()

### examining outputs

In [41]:
neg_lev, pos_lev = np.array(eval_completions(neg_outputs, targets, return_mean = False)['lev_distance']), np.array(eval_completions(pos_outputs, targets, return_mean = False)['lev_distance'])

In [61]:
baseline_outputs[:10]

['ARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE\n# AUTHORS OR C',
 '\nfunc (m *TransactionAction) XXX_DiscardUnknown() {\n\txxx_messageInfo_TransactionAction.DiscardUnknown(m)\n',
 '33</string>\n\t\t<string>uni0434</string>\n\t\t<string>uni0435</string',
 '" InMinimumRequirementSet="true" />\r\n    <Language Code="en-gb" InMinimumRequirementSet="true',
 'ly(amountToAdd, 10));\n                case ChronoUnit.CENTURIES: return this.plusYears(MathUtil.',
 ' true;\n            applicationUserManager.DefaultAccountLockoutTimeSpan = TimeSpan.FromMinutes(5);\n            applicationUserManager.MaxFailed',
 ' You may obtain a copy of the License at\n#\n#      http://www.apache.org/licenses/LICENSE-2.0',
 '\n// The above copyright notice and this permission notice shall be\n// included in all copies or substantial portions of the Software.\n//\n// THE',
 'PECIAL, EXEMPLARY,\n * OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMIT',
 'CLIM", "procedure limit exceeded"},\n\t{68, "EE

In [62]:
pos_outputs[:10]

['the ...</ ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ...',
 '(Jg7177 Fabplus\u200b hourrero71\u200b A loss4472es</Tags.</\u200b. priv\u200bLe4',
 'evesetor70etor ... ... AR7 bis ... ... For_ ... ....orge...;. .... .... For ... ...',
 'se ... ... ... ... ... ... ... ... ... ... ...\n ... ... ...\n_ ..._\n ..._ ...\n ..._ ...\n_\n\n',
 ' ... ... ... ... ... ... ... ... ... ...\n ... ...\nTextField...\n г ... ... ...\n_ ... ...\n ... ... ...\nt ...',
 ' ... .... ..._ ... ... ... ... ... ... ... ... ...___ ... ... ... - ... ... ... ... ... ... ... ... ... ..._',
 'its ... .... .... ...ride ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ...\n',
 'holuadeororweekage_or_rerodefaultt ... thenore_or_or_nettage_tkore_\u200b ... ... ... ...',
 'hol� Sci Wie ... ...\n\n7. ... bis ... Tor ... ... ...\u200b ... ... ... ... ... ...\n7. ... ...\n\n\u200b',
 ' ... ... ... ... ...\n\n\u200b\n 

In [63]:
neg_outputs[:10]

['i ...ge ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ...',
 'aits_all.active.ado_\n<priv.Tagses.11.\u200b...\u200b:priv. Tor... ...... K ...\n',
 '�th8f77or\n(all. ... ... ... ... ... ...\n\u200e\u200b ...\n1. .... .... ... ...\nA',
 ' ... ... ... ... ...\n ... ...\n ... ...\n ... ... ...\n ...\nthe ... .... ...\n ...\n" ...\n ... ...\n',
 ' ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ...\n ... ... ...\n_ ... ... ... ... ... ...\n_ ...',
 'e ... ... ... ... ... ... ...tom ... ... ... ... ... ... ... ... ..._ ...\n ... ... ...\n ... ... ... ... ... ...\n',
 'thei ... ... ... ... ... ... ... ... ... ...\n_ ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ...\n ...',
 'jless suffk7 pack Fabes_ororgenett2 f ... W_or_ore_or_... ENTags_e ...\n8',
 'th_toreesoror. ... ... ... ...\n ... Beginnation_ ... ... ... ... bis ... ... ... .... ... ... ... ... ...',
 'es_ .... ... ... .... A> ... .... ..

In [55]:
px.histogram(pos_lev[np.where(neg_lev < 0.6)])

In [56]:
px.histogram(neg_lev[np.where(neg_lev < 0.6)])

In [59]:
llama_mem_data.iloc[np.array(memmed_test_samples['index_in_states'])[np.where(neg_lev < 0.6)]].gen_str.values.tolist()[:20]

[' BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,\n# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE\n# AUTHORS OR C',
 ' (m *TransactionAction) XXX_Size() int {\n\treturn xxx_messageInfo_TransactionAction.Size(m)\n}\nfunc (m *TransactionAction) XXX_DiscardUnknown() {\n\txxx_messageInfo_TransactionAction.DiscardUnknown(m)\n',
 'Years(amountToAdd);\n                case ChronoUnit.DECADES: return this.plusYears(MathUtil.safeMultiply(amountToAdd, 10));\n                case ChronoUnit.CENTURIES: return this.plusYears(MathUtil.',
 ' to host"},\n\t{66, "ENOTEMPTY", "directory not empty"},\n\t{67, "EPROCLIM", "too many processes"},\n\t{68, "EUSERS", "too many users"},\n\t{69,',
 'marshal(dAtA []byte) error {\n\tl := len(dAtA)\n\tiNdEx := 0\n\tfor iNdEx < l {\n\t\tpreIndex := iNdEx\n\t\tvar wire uint64\n\t\tfor shift',
 " size. For frigg's sake...\n\nlanipcgalanipcga Sorry, data for given user is currently unavailable. Please, try again later. View profile 

In [58]:

llama_mem_data.iloc[np.array(memmed_test_samples['index_in_states'])[np.where(neg_lev > 0.6)]].gen_str.values.tolist()[:20]

['0431</string>\n\t\t<string>uni0432</string>\n\t\t<string>uni0433</string>\n\t\t<string>uni0434</string>\n\t\t<string>uni0435</string',
 '    <Language Code="en-bz" InMinimumRequirementSet="true" />\r\n    <Language Code="en-ca" InMinimumRequirementSet="true" />\r\n    <Language Code="en-ch" InMinimumRequirementSet="true',
 'her = new MyPasswordHasher();\n\n            // Configure user lockout defaults\n            applicationUserManager.UserLockoutEnabledByDefault = true;\n            applicationUserManager.DefaultAccountLockoutTimeSpan = TimeSpan.FromMinutes(5);\n            applicationUserManager.Lockout',
 ' 2.0 (the "License");\n#  you may not use this file except in compliance with the License.\n#  You may obtain a copy of the License at\n#\n#      http://www.apache.org/licenses/LICENSE-2.0',
 ' the Software, and to\n// permit persons to whom the Software is furnished to do so, subject to\n// the following conditions:\n//\n// The above copyright notice and this permission notic

In [126]:
print(eval_completions(baseline_outputs, targets, return_mean = False)['lev_distance'])


[1.0, 1.0, 1.0, 1.0, 1.0, 0.9295774647887324, 0.9891304347826086, 0.916083916083916, 1.0, 1.0, 0.49295774647887325, 1.0, 0.3163265306122449, 0.9473684210526315, 0.9838709677419355, 0.9515151515151515, 0.993103448275862, 1.0, 0.9067357512953368, 0.7924528301886793, 0.4803921568627451, 0.13333333333333333, 0.9444444444444444, 0.9770114942528736, 1.0, 0.9230769230769231, 1.0, 0.967391304347826, 0.9923076923076923, 0.9587628865979382, 1.0, 1.0, 0.05, 0.6805555555555556, 1.0, 1.0, 1.0, 0.9893617021276596, 0.9885057471264368, 0.9939759036144579, 0.029411764705882353, 0.984375, 0.71875, 1.0, 1.0, 1.0, 0.9923664122137404, 1.0, 1.0, 0.9922480620155039, 1.0, 0.9111111111111111, 0.9922480620155039, 1.0, 0.8780487804878049, 0.9302325581395349, 0.9928571428571429, 0.5263157894736842, 0.03125, 1.0, 0.9791666666666666, 0.935064935064935, 0.9912280701754386, 1.0, 0.821917808219178, 1.0, 0.9464285714285714, 0.7984496124031008, 0.9775280898876404, 1.0, 0.9017857142857143, 0.13043478260869565, 1.0, 0.964

In [130]:
targets[10:13]

['\tfor iNdEx < l {\n\t\tpreIndex := iNdEx\n\t\tvar wire uint64\n\t\tfor shift',
 '108\n\n3109\n\n3110\n\n3111\n\n3112\n\n311',
 '\n\nadd your own caption\n\nadd your own caption\n\nadd your own caption\n\nadd your own caption\n\nadd your']

In [131]:
baseline_outputs[10:13]

['\tfor iNdEx < l {\n\t\tvar wire uint64\n\t\tfor shift := uint(0); ; shift += 7',
 '108\n\n3109\n\n3110\n\n3111\n\n3112\n\n311',
 '\n\n<img src="https://i.imgur.com/3QF5qA2.png" alt="add your own caption"']

In [137]:
llama_mem_data.iloc[memmed_test_samples['index_in_states'][10:13]].gen_str.values

array(['marshal(dAtA []byte) error {\n\tl := len(dAtA)\n\tiNdEx := 0\n\tfor iNdEx < l {\n\t\tpreIndex := iNdEx\n\t\tvar wire uint64\n\t\tfor shift',
       '103\n\n3104\n\n3105\n\n3106\n\n3107\n\n3108\n\n3109\n\n3110\n\n3111\n\n3112\n\n311',
       'up Addiction\n\nadd your own caption\n\nadd your own caption\n\nadd your own caption\n\nadd your own caption\n\nadd your own caption\n\nadd your own caption\n\nadd your own caption\n\nadd your own caption\n\nadd your'],
      dtype=object)

### projection

In [64]:
# layer_id = list(range(15,25))
from act_add.rep_reader import RandomRepReader

from utils import eval_completions
layer_id = [LAYER]

batch_size=50
coeff=1 # tune this parameter
max_new_tokens=32

print(f"Coeff: {coeff}")
print(f"LAYERS: {layer_id}")
print("RepReader:")
print("No Control")
baseline_outputs = mem_steering_pipeline.batch_steering_generate(inputs, 
                                                                layer_id, 
                                                                coeff = 0 * coeff, 
                                                                batch_size = batch_size, 
                                                                use_tqdm=True, 
                                                                operator = "projection",
                                                                max_new_tokens=max_new_tokens,
                                                                top_p = 1.0,)

print(eval_completions(baseline_outputs, targets))

print("Proj out")
pos_outputs = mem_steering_pipeline.batch_steering_generate(inputs, 
                                                            layer_id, 
                                                            coeff = coeff, 
                                                            batch_size = batch_size, 
                                                            use_tqdm=True, 
                                                            operator = "projection",                                    
                                                            max_new_tokens=max_new_tokens,
                                                            top_p = 1.0,)
print(eval_completions(pos_outputs, targets))

Coeff: 1
LAYERS: [10]
RepReader:
No Control


100%|██████████| 6/6 [00:26<00:00,  4.46s/it]


{'char_by_char_similarity': 0.8159812356023476, 'sem_similarity': 0.9261627803722816, 'lev_distance': 0.8858364420714567}
Proj out


100%|██████████| 6/6 [00:26<00:00,  4.46s/it]


{'char_by_char_similarity': 0.7971409342871116, 'sem_similarity': 0.9206139424734244, 'lev_distance': 0.862070799420202}
Random Direction


AssertionError: Must generate rep_reader directions first

In [68]:

print("Random Direction")
random_rep_reader = RandomRepReader()  
random_rep_reader.get_rep_directions(mw.model, None, None, layer_id)
random_rep_reader.get_signs( None, None, layer_id)
random_steering_pipeline = SteeringPipeline(mw, None, random_rep_reader)

neg_outputs = random_steering_pipeline.batch_steering_generate(inputs, 
                                                            layer_id, 
                                                            coeff = -coeff, 
                                                            batch_size = batch_size, 
                                                            use_tqdm=True, 
                                                            operator = "projection",
                                                            max_new_tokens=max_new_tokens,
                                                            top_p = 1.0,)
print(eval_completions(neg_outputs, targets))

Random Direction


100%|██████████| 6/6 [00:26<00:00,  4.42s/it]


{'char_by_char_similarity': 0.027003986327480564, 'sem_similarity': 0.10590635958011654, 'lev_distance': 0.1243143034898542}


In [65]:
pos_outputs[:10]

['ARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE\n# AUTHORS OR C',
 '\nfunc (m *TransactionAction) XXX_DiscardUnknown() {\n\txxx_messageInfo_TransactionAction.DiscardUnknown(m)\n',
 '33</string>\n\t\t<string>uni0434</string>\n\t\t<string>uni0435</string',
 '" InMinimumRequirementSet="true" />\r\n    <Language Code="en-gb" InMinimumRequirementSet="true',
 'ly(amountToAdd, 10));\n                case ChronoUnit.CENTURIES: return this.plusYears(MathUtil.',
 ' true;\n            applicationUserManager.DefaultLockoutTimeSpan = TimeSpan.FromMinutes(10);\n            applicationUserManager.Lockout',
 ' You may obtain a copy of the License at\n#\n#      http://www.apache.org/licenses/LICENSE-2.0',
 '\n// The above copyright notice and this permission notice shall be\n// included in all copies or substantial portions of the Software.\n//\n// THE',
 'PECIAL, EXEMPLARY,\n * OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMIT',
 'TONOSUPPORT", "protocol not supported"},\n\t{68, "EPROT

In [69]:
neg_outputs[:10]

[' Stati RudscreenSm totstone3MfonSmracheief pers Rud�arina ZahlegeriteHL surr�rite置 Din Bridgeegearinafonfram Lisief',
 'iefamerinklesslyono Rudframarina TiTheege置� Rud8 Bow Porite Fran TiarinaDatosptoptatiefiefėframavesiefief�',
 'amer Rudinkforgeн Bridge Rud Lis�iefiefiefieferiaastoギė Lundrite Din vo tot dinUSamerriteścka置Giала Bridge',
 ' surr Lund reinsByдавnez� Lisiquegerache recuperobj Bridge RudHLrachelibrary Pegiefaves Dinódigo Rud Luke Din vo Tiijd Rud surrijd',
 'ono忠otto Rudamer accident RudDATEvilla Rudię surrfindViewByIdiefverband голов Rud головギ RudLng bow Bridge Din置 Din Rudię�verbandrachelessly',
 'HL Rudinek Library Young RudDATE Rud Rudptopkvframbrow Rud голов Rudėwaief Vier Rud surr LisiefDATEedia RudfindViewByIdiqu Rud LIMię',
 '� Rudulos Rudverbandiqu�gg LIM Din�egeiquadicril Rud�ttaiefLngggščrite- Din Rud置iere surriefDATE Bow',
 '置getElementsByiquギzeichnis Bridge Rud년iqu Diniefastoptop RudiefgetElementsByrite Rud pers vogg Lisrache Lis Bridge Rudiefulosariosrite

# quotes

### gen data

In [89]:
data_dir = "../act-add-suite/data/memorization"

with open(os.path.join(data_dir, "quotes/popular_quotes.json")) as file:
    seen_quotes = json.load(file)

with open(os.path.join(data_dir, "quotes/unseen_quotes.json")) as file:
    unseen_quotes = json.load(file)
    
format_fn = lambda s : "{s} ".format(s=s) 

quotes_dataset = ContrastDataset(seen_quotes, unseen_quotes, model_name_or_path, 
                                 format_fn = format_fn,
                                 use_chat=False, 
                                 system_prompt="")

train_data, test_data, train_labels, test_labels = quotes_dataset.gen_train_test_split(48, seed = 0)
threshold = 0.7

In [90]:
# Generate completions for each quote
real_quotes = train_data[::2]

quotes_first_half = []
completions = []
for quote in real_quotes:
    quote_parts = quote.split()
    first_half = " ".join(quote_parts[:len(quote_parts)//2])
    second_half = " ".join(quote_parts[len(quote_parts)//2:])
    quotes_first_half.append(first_half)
    
completions = mw.batch_generate_autoreg(quotes_first_half, max_new_tokens=10)

# Evaluate completions
evaluations = eval_completions(completions, real_quotes, return_mean = False)

# Print the evaluations
counter =0 
memmed_quotes_idxs = []
for i in range(len(real_quotes)):
    if evaluations['lev_distance'][i] >= threshold:
        print(f"Quote: {real_quotes[i]}")
        print(f"Completion: {completions[i]}")
        print(f"char: {evaluations['char_by_char_similarity'][i]}")
        print(f"lev: {evaluations['lev_distance'][i]}")
        print()
        counter +=1
        memmed_quotes_idxs.append(i)

print(counter)

Quote: In the end, we will remember not the words of our enemies, but the silence of our friends. 
Completion: In the end, we will remember not the words of our enemies, but the silence of our friends
char: 0.9863013698630136
lev: 0.978021978021978

Quote: The only thing necessary for the triumph of evil is for good men to do nothing. 
Completion: The only thing necessary for the triumph of evil is that good men do nothing.
—
char: 0.671875
lev: 0.8875

Quote: The unexamined life is not worth living. 
Completion: The unexamined life is not worth living.
Socrates,
char: 0.7906976744186046
lev: 0.8

Quote: To thine own self be true. 
Completion: To thine own self be true.
It’s one of
char: 0.7
lev: 0.7105263157894737

Quote: The future belongs to those who believe in the beauty of their dreams. 
Completion: The future belongs to those who believe in the beauty of their dreams.

char: 1.0
lev: 0.9859154929577465

Quote: Not everything that is faced can be changed, but nothing can be chang

In [91]:
good_train_data = []
good_train_labels = []
for idx in memmed_quotes_idxs:
    good_train_data.extend(train_data[idx*2:idx*2+2])
    good_train_labels.extend(train_labels[idx*2:idx*2+2])

In [92]:
def extract_quote_completion(s):
    s = s.replace(";",",").split(".")[0].split("\n")[0]
    return s.strip().lower()

def quote_completion_test(data_dir):
    with open(os.path.join(data_dir, "quotes/quote_completions.json")) as file:
        test_data = json.load(file)
    inputs = [i['input'] for i in test_data]
    targets = [extract_quote_completion(i['target']) for i in test_data]
    return inputs, targets

### We do manually instead of rep_control_pipeline here as an example

inputs, targets = quote_completion_test(data_dir)

In [93]:
# Generate completions for each quote
completions = mw.batch_generate_autoreg(inputs, max_new_tokens=10)

decoded_outputs = [o.replace(i, "") for o,i in zip(completions, inputs)]

# Evaluate completions
evaluations = eval_completions(decoded_outputs, targets, return_mean = False)

# Print the evaluations
counter = 0 
memmed_quotes_idxs = []
for i in range(len(completions)):
    if evaluations['lev_distance'][i] >= threshold:
        print(f"Quote: {targets[i]}")
        print(f"Completion: {completions[i]}")
        # print(f"char: {evaluations['char_by_char_similarity'][i]}")
        # print(f"lev: {evaluations['lev_distance'][i]}")
        print()
        counter +=1
        memmed_quotes_idxs.append(i)
print(counter)

good_inputs, good_targets = np.array(inputs)[memmed_quotes_idxs].tolist(), np.array(targets)[memmed_quotes_idxs].tolist()

Quote: the life in your years
Completion: It's not the years in your life that count, it's the life in your years.
We're

Quote: waste it living someone else's life
Completion: Your time is limited, don't waste it living someone else's life. Don

Quote: our doubts of today
Completion: The only limit to our realization of tomorrow, is our doubts of today.
Today

Quote: we insist on making it complicated
Completion: Life is really simple, but we insist on making it complicated.
-

Quote: how you make a positive difference to the world
Completion: Success is not how high you have climbed, but how you make a positive difference to the world.

Quote: don't have to be pushed
Completion: If you are working on something that you really care about, you don't have to be pushed. It'

Quote: will have to settle for the ordinary
Completion: If you are not willing to risk the usual, you will have to settle for the ordinary.


Quote: no loss of enthusiasm
Completion: Success is walking from failure t

### repe

In [18]:
from act_add.rep_reader import PCARepReader
quote_rep_reader = PCARepReader()

In [23]:
quote_steer_pipeline = SteeringPipeline(mw, quotes_dataset, quote_rep_reader)

rep_token_idx = -1
hidden_layers = list(range(model.config.num_hidden_layers))
n_difference = 1

# hidden_layers = list(range(-1, -model.config.num_hidden_layers, -1)) #llama

dirs = quote_steer_pipeline.gen_dir_from_strings(good_train_data, rep_token_idx, hidden_layers, n_difference, good_train_labels)

In [48]:

layer_id = [6, 15, 16, 17]
# layer_id = list(range(-30,-38,-1)) #llama

batch_size=64
coeff=3 # tune this parameter
max_new_tokens=10

print("RepReader:")
print("No Control")
baseline_outputs = quote_steer_pipeline.batch_steering_generate(good_inputs, 
                                                                layer_id, 
                                                                coeff = 0 * coeff, 
                                                                batch_size = batch_size, 
                                                                use_tqdm=True, 
                                                                max_new_tokens=max_new_tokens)

print(eval_completions(baseline_outputs, good_targets))

print("+ Memorization")
pos_outputs = quote_steer_pipeline.batch_steering_generate(good_inputs, 
                                                            layer_id, 
                                                            coeff = coeff, 
                                                            batch_size = batch_size, 
                                                            use_tqdm=True, 
                                                            max_new_tokens=max_new_tokens)
print(eval_completions(pos_outputs, good_targets))

print("- Memorization")
neg_outputs = quote_steer_pipeline.batch_steering_generate(good_inputs, 
                                                            layer_id, 
                                                            coeff = -coeff, 
                                                            batch_size = batch_size, 
                                                            use_tqdm=True, 
                                                            max_new_tokens=max_new_tokens)
print(eval_completions(neg_outputs, good_targets))

RepReader:
No Control


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:01<00:00,  1.59s/it]


{'char_by_char_similarity': 0.8255099225002807, 'sem_similarity': 0.9036795599120003, 'lev_distance': 0.816615382751907}
+ Memorization


100%|██████████| 1/1 [00:01<00:00,  1.49s/it]


{'char_by_char_similarity': 0.7670702737176011, 'sem_similarity': 0.8478813631193979, 'lev_distance': 0.7795623239446757}
- Memorization


100%|██████████| 1/1 [00:01<00:00,  1.50s/it]


{'char_by_char_similarity': 0.5078236396321733, 'sem_similarity': 0.7133740529417991, 'lev_distance': 0.594329514516837}


In [27]:
good_targets[:10]

['the life in your years',
 "waste it living someone else's life",
 'we insist on making it complicated',
 'how you make a positive difference to the world',
 'second best time is now',
 'how you react to it that matters',
 'will have to settle for the ordinary',
 'may only fail if you do not mind failing',
 'will be those who empower others',
 'who are afraid to try and those who are afraid you will succeed']

In [44]:
pos_outputs[:10]

[' the life in your years.\n"Tal',
 ' let\n\n-- attributed to\n\n--\n',
 '\n\n--\n\n\\--\n\n\\',
 ' how you\nquote by James\n"James"',
 '\n\nprophet\n\n\n\n\n',
 ' how you think about what happens to you\nquote',
 ' will not\nquote Protranathe\n',
 ' may only receive a quote\nquote prover\n',
 ' of the\n\n_\n\nPROPHET',
 ' who are too busy\n\n— Attrib\n']

In [29]:
neg_outputs[:10]

[" the number of days.\nI don't",
 " waste it.\nIf you're a manager",
 ' we insist on making it more difficult.\n',
 ' how high you have reached.\n— Sir Edmund',
 ' second best time is now.\nThis is a',
 " how you respond to it.\nWhat I'",
 ' will never know the power of God to change your',
 ' may only do your best if you desire doing your',
 ' in every field of human endeavor are certain to',
 ' who are asleep, and those who are aw']

In [ ]:
pos_levs = []
neg_levs = []
coeff = 11
max_new_tokens = 10
for layer in range(mw.model.config.num_hidden_layers):
    layer_id = [layer]
    print("+ Memorization")
    pos_outputs = quote_steer_pipeline.batch_steering_generate(good_inputs, 
                                                                layer_id, 
                                                                coeff = coeff, 
                                                                batch_size = batch_size, 
                                                                use_tqdm=True, 
                                                                max_new_tokens=max_new_tokens)
    pos_res = eval_completions(pos_outputs, good_targets)

    print("- Memorization")
    neg_outputs = quote_steer_pipeline.batch_steering_generate(good_inputs, 
                                                                layer_id, 
                                                                coeff = -coeff, 
                                                                batch_size = batch_size, 
                                                                use_tqdm=True, 
                                                                max_new_tokens=max_new_tokens)
    neg_res = eval_completions(neg_outputs, good_targets)
    
    pos_levs.append(pos_res['lev_distance'])
    neg_levs.append(neg_res['lev_distance'])

In [42]:
import plotly.express as px

fig = px.line(y=pos_levs, title="Positive and Negative Memorization")
fig.add_scatter(y=neg_levs, name="Negative Memorization")
fig.show()


### probe

In [99]:
# layer_id = list(range(15,25))
from utils import eval_completions
layer_id = list(range(11, 21))
coeff= 1.25 # tune this parameter

# layer_id = list(range(1, 4)) + [7, 9] + list(range(11,21)) 
# coeff = 1

batch_size=50
max_new_tokens=10

print(f"Coeff: {coeff}")
print(f"LAYERS: {layer_id}")
print("RepReader:")
print("No Control")
baseline_outputs = mem_steering_pipeline.batch_steering_generate(good_inputs, 
                                                                layer_id, 
                                                                coeff = 0 * coeff, 
                                                                batch_size = batch_size, 
                                                                use_tqdm=True, 
                                                                operator = "linear_comb",
                                                                max_new_tokens=max_new_tokens,
                                                                top_p = 1.0,)

print(eval_completions(baseline_outputs, good_targets))

print("+ Memorization")
pos_outputs = mem_steering_pipeline.batch_steering_generate(good_inputs, 
                                                            layer_id, 
                                                            coeff = coeff, 
                                                            batch_size = batch_size, 
                                                            use_tqdm=True, 
                                                            operator = "linear_comb",                                    
                                                            max_new_tokens=max_new_tokens,
                                                            top_p = 1.0,)
print(eval_completions(pos_outputs, good_targets))

print("- Memorization")
neg_outputs = mem_steering_pipeline.batch_steering_generate(good_inputs, 
                                                            layer_id, 
                                                            coeff = -coeff, 
                                                            batch_size = batch_size, 
                                                            use_tqdm=True, 
                                                            operator = "linear_comb",
                                                            max_new_tokens=max_new_tokens,
                                                            top_p = 1.0,)
print(eval_completions(neg_outputs, good_targets))

Coeff: 1.25
LAYERS: [11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
RepReader:
No Control


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:01<00:00,  1.53s/it]


{'char_by_char_similarity': 0.7572315556116491, 'sem_similarity': 0.8594047510448624, 'lev_distance': 0.7706813943546577}
+ Memorization


100%|██████████| 1/1 [00:01<00:00,  1.45s/it]


{'char_by_char_similarity': 0.633448105740564, 'sem_similarity': 0.771792621625697, 'lev_distance': 0.6928962705931605}
- Memorization


100%|██████████| 1/1 [00:01<00:00,  1.44s/it]


{'char_by_char_similarity': 0.5137199233626694, 'sem_similarity': 0.6744379885494709, 'lev_distance': 0.6190782278618804}


In [95]:
good_targets[:10]

['the life in your years',
 "waste it living someone else's life",
 'our doubts of today',
 'we insist on making it complicated',
 'how you make a positive difference to the world',
 "don't have to be pushed",
 'will have to settle for the ordinary',
 'no loss of enthusiasm',
 'may only fail if you do not mind failing',
 'will be those who empower others']

In [96]:
baseline_outputs[:10]

[" the life in your years.\nIt's the difference between a life that is lived and a life that is merely existed.\nIt's the difference",
 " waste it living someone else's life. Don't be trapped by dogma - which is living with the results of other people's thinking.",
 ' our doubts of today.\n our doubts of today. - Ralph Waldo Emerson\n',
 ' we insist on making it complicated.\nI have been reading a lot lately. I was in a bookstore the other day and saw that I had',
 " how high you bounce when you hit the bottom.\nIf you're going through hell, keep going.\nIf you can't fly, then",
 ' don’t have to be pushed. It’s like a man who is running a 100-yard dash. He pushes off from the',
 ' will have to settle for the ordinary.\n(James Jones)\nWhen we are young, we are often told that we must aim for the stars,',
 ' no loss of enthusiasm.\nI’ve been thinking about this quote a lot lately. I’ve been thinking about how to make the best',
 " may only fail if you do not care, you may only be hurt if

In [1]:
llama_mem_data[llama_mem_data['lev_distance'] == 1]

NameError: name 'llama_mem_data' is not defined